In [4]:
#python packages pd
import numpy as np
import matplotlib.pyplot as plt




#machine learning packages

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split



import importlib

#custom python scripts
import generator

import utilis

In [11]:
# Check that you are running GPU's
utilis.GPU_checker()

InvalidArgumentError: device CUDA:0 not supported by XLA service
	while setting up XLA_GPU_JIT device number 0

In [9]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7ffae4703ac8>


In [12]:
%%time
#  generators
importlib.reload(generator)
training_generator = generator.Keras_DataGenerator( dataset='train', w_hyp=False)
validation_generator = generator.Keras_DataGenerator(dataset='valid', w_hyp= False)

Retrieving data from deephol-data-processed/proofs/human/train/
Generating examples from a set of 376968 examples
Retrieving data from deephol-data-processed/proofs/human/valid/
Generating examples from a set of 104054 examples
CPU times: user 2.33 s, sys: 1.26 s, total: 3.59 s
Wall time: 10.4 s


In [17]:
#Constants
ARE YOU LOADINNG MODE?
VOCAB_SIZE = 1254
INPUT_LENGTH = 1000
EMBEDDING_DIM = 128

# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(CuDNNLSTM(128)))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Object `MODE` not found.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 128)         160512    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1000, 128)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               264192    
_________________________________________________________________
dense_2 (Dense)              (None, 41)                10537     
Total params: 435,241
Trainable params: 435,241
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
ARE YOU LOADINNG MODE

In [6]:
## WARNING IF YOU CONTAIN MULTIPLE CORE GPUS
# NOTE unclear if these causes a speed up 
# @TANCREDI, I HAVE TREID THIS ON JUST GOALS AND DOES NOT SEEM TO CAUSE A SPEED UP MAY 
#CAUSE A SPEED UP IF WE USE HYPOTHESIS
# unclea rif this seepd 
# from keras.utils import multi_gpu_model
# model_GPU = multi_gpu_model(model, gpus= 4)
#model_GPU.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
## ARE YOU LOADING A MODEL IF YES RUN TEH FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
#overwriting model
model = loaded_model

In [10]:
 print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 128)         160512    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 128)         0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dense_1 (Dense)              (None, 41)                5289      
Total params: 297,897
Trainable params: 297,897
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
%%time
n_epochs = 2
history = model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        verbose=1,
                        use_multiprocessing= False,
                        epochs=n_epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/2


InvalidArgumentError: device CUDA:0 not supported by XLA service
	while setting up XLA_GPU_JIT device number 0

In [32]:
# FOR SAVING MODEL
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [7]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "SIMPLE_LSTM_SMALL_TANK"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [33]:
# save history - WARNING FILE NAME 
utilis.history_saver_bad(history, LOSS_FILE_NAME)

SAVED SOME LOGS -- OVERWROTE OLD LOGS -- SOMEONE NEEDS TO FIX THIS


In [ ]:
# read numpy array
# history_toplot = np.genfromtxt("training_logs/"+ LOSS_FILE_NAME +".csv")
# plt.plot(history_toplot)
# plt.title('Loss history')
# plt.show()

Saved model to disk


Loaded model from disk


In [27]:
%%time
n_epochs = 1
history = loaded_model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        verbose=1,
                        use_multiprocessing= False,
                        epochs=n_epochs)

Epoch 1/1
  39/5814 [..............................] - ETA: 22:12 - loss: 2.7200 - acc: 0.1775

KeyboardInterrupt: 